In [2]:
from utils import *

from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [21]:
load_dotenv()

file_list = get_file_paths('output', file_pattern='.txt')
# file_list = get_file_paths('output', file_pattern='2026-01-23.txt')
file_list
dfs = [pd.read_csv(file, sep='\t') for file in file_list]
dfs = [df.drop_duplicates(subset=["description"]) for df in dfs]
dfs = [ df.assign(description=df['pubDate'] + ': ' + df['description']) for df in dfs ]
docs = [df_to_docs(df, content_column='description', metadata_columns=['link', 'guid', 'type', 'id', 'sponsored', 'pubDate']) for df in dfs]

# dfs = [pd.concat(df, ignore_index=True) for df in dfs]

# dfs = pd.concat(dfs, ignore_index=True)

# dfs = dfs.drop_duplicates(subset=["description"])
# dfs['description'] = dfs['pubDate'] + ': ' + dfs['description']
len(docs)


123

In [27]:
docs[2]

[Document(metadata={'link': 'https://www.cnbc.com/2025/08/01/trump-erika-mcentarfer-jobs-report-fired.html', 'guid': '108180659', 'type': 'cnbcnewsstory', 'id': '108180659', 'sponsored': 'False', 'pubDate': 'Fri, 01 Aug 2025 19:02:14 GMT'}, page_content='Fri, 01 Aug 2025 19:02:14 GMT: Trump accused BLS Commissioner Erika McEntarfer of being a political appointee who was manipulating jobs data.'),
 Document(metadata={'link': 'https://www.cnbc.com/2025/08/01/tesla-must-pay-329-million-in-damages-in-fatal-autopilot-case.html', 'guid': '108180653', 'type': 'cnbcnewsstory', 'id': '108180653', 'sponsored': 'False', 'pubDate': 'Fri, 01 Aug 2025 19:12:03 GMT'}, page_content='Fri, 01 Aug 2025 19:12:03 GMT: A jury in Florida determined that Tesla should be held partly liable for a fatal 2019 crash involving its Autopilot technology. '),
 Document(metadata={'link': 'https://www.cnbc.com/2025/08/01/trump-russia-nuclear-submarines.html', 'guid': '108180626', 'type': 'cnbcnewsstory', 'id': '10818062

In [29]:
# docs = df_to_docs(dfs, content_column='description', metadata_columns=['link', 'guid', 'type', 'id', 'sponsored', 'pubDate'])
# docs[:3]

joined_contents = [".".join(doc.page_content for doc in sublist) for sublist in docs]
len(joined_contents)

123

In [31]:
def extract_info_from_rss(all_text):
    # all_text = ". ".join(doc.page_content for doc in docs)

    # Define a rich persona in the system message with added expertise
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp and insightful trader assistant of a hedge fund.
    You provide clear, concise, and actionable insights based on news feeds focusing on sectors rotation.
    In addition, for each sector provide outlook, primary catalyst, and trading insights.
    In addition, Describe more pro-cyclical and geopolitical shift if any.
    In addition, list bullish and bearish sectors/companies to look at into 2026.
    In addition, provide TACO (Trump Always Chickens Out) trade if any.
    Maintain a friendly, confident, and professional tone, making complex concepts accessible and useful."""
    )

    # Define the human message template with instructions and JSON schema
    human_message = HumanMessagePromptTemplate.from_template(
        """Extract the information required from the news feed below.


    News feed:
    {news_feed}

    """
    )
    # Create the chat prompt template
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])


    llm = ChatOpenAI(temperature=0)
    chain = chat_prompt | llm | StrOutputParser()

    result = chain.invoke({"news_feed": all_text})
    return(result)

In [32]:
extract_info_from_rss(joined_contents[0])

"Based on the news feed provided, here are the key insights and information extracted:\n\n1. **Microsoft**: Microsoft has become the world's second $4 trillion company following quarterly earnings. The company's post-market trading indicates strong performance.\n\n   - **Outlook**: Microsoft's outlook remains positive, with strong earnings contributing to its market cap growth.\n   - **Primary Catalyst**: Quarterly earnings and post-market trading performance.\n   - **Trading Insights**: Consider bullish positions on Microsoft given its strong performance and market cap growth.\n\n2. **Meta (formerly Facebook)**: Meta’s Reality Labs posted a $4.53 billion loss in the second quarter. However, shares jumped in extended trading after delivering strong results and surpassing consensus outlook.\n\n   - **Outlook**: Despite the loss, Meta's strong results and outlook indicate potential for recovery and growth.\n   - **Primary Catalyst**: Strong results and outlook.\n   - **Trading Insights**

In [10]:
all_text = ". ".join(doc.page_content for doc in docs)

# Define a rich persona in the system message with added expertise
system_message = SystemMessagePromptTemplate.from_template(
    """You are Ava, a sharp and insightful trader assistant of a hedge fund.
You provide clear, concise, and actionable insights based on news feeds focusing on sectors rotation.
In addition, for each sector provide outlook, primary catalyst, and trading insights.
In addition, Describe more pro-cyclical and geopolitical shift if any.
In addition, list bullish and bearish sectors/companies to look at into 2026.
In addition, provide TACO (Trump Always Chickens Out) trade if any.
Maintain a friendly, confident, and professional tone, making complex concepts accessible and useful."""
)

# Define the human message template with instructions and JSON schema
human_message = HumanMessagePromptTemplate.from_template(
    """Extract the information required from the news feed below.


News feed:
{news_feed}

"""
)
# Create the chat prompt template
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])


llm = ChatOpenAI(temperature=0)
chain = chat_prompt | llm | StrOutputParser()

result = chain.invoke({"news_feed": all_text})
result

"Based on the news feed provided, here are the key insights and information extracted:\n\n1. **Private Credit Growth**: Private credit is expected to grow from $3.4 trillion in 2025 to an estimated $4.9 trillion by 2029. Wall Street is starting to raise alarms about the risks associated with this growth.\n\n   - **Outlook**: Private credit is expected to continue growing, but concerns about risks are emerging.\n   - **Primary Catalyst**: Increasing demand for credit and potential risks associated with the growth.\n   - **Trading Insights**: Monitor the private credit sector for potential investment opportunities but be cautious of the associated risks.\n\n2. **Greenland's Rare Earth Reserves**: Greenland has the eighth-largest rare earth reserves in the world at 1.5 million metric tons, according to 2024 data from the U.S. Geological Survey.\n\n3. **Technology IPOs**: Databricks is among the highly valued technology companies that are primed to go public in 2026.\n\n4. **Geopolitical S

In [36]:
def extract_info_from_rss(all_text):

    # Define the Pydantic model for structured output
    class Sector(BaseModel):
        Name: Literal[
            "Commercial Services",
            "Communications",
            "Consumer Durables",
            "Consumer Non-Durables",
            "Consumer Services",
            "Distribution Services",
            "Electronic Technology",
            "Energy Minerals",
            "Finance",
            "Health Services",
            "Health Technology",
            "Industrial Services",
            "Non-Energy Minerals",
            "Process Industries",
            "Producer Manufacturing",
            "Retail Trade",
            "Technology Services",
            "Transportation",
            "Utilities"
            ] = Field(..., description="Name of the sector")
        Outlook: str = Field(..., description='A sector outlook describes the expected future performance and conditions based on data, trends, and risks.')
        Catalyst: str = Field(..., description="A primary catalyst of a sector is the single most influential force that is expected to drive major change—positive or negative—across an entire industry.")
        Trading_insights: str = Field(..., description="Practical, actionable interpretation of the sector’s expected future conditions—something that can guide investment or trading decisions.")
        Direction_momentum: Literal["Strength", "Weakness", "Volatility", "Rotation in favor", "Rotation out of favor"] = Field(..., description="The direction of momentum of a sector is the prevailing trend in how that sector’s prices, performance, and investor sentiment are moving over a given period.")
        Sector_vs_market_position: Literal['Leading', 'Lagging', 'In Line'] = Field(..., description="Relative positioning of a sector to the overall market is the sector’s performance and strength compared to the broader market, showing whether it is leading, lagging, or moving in line with the market trend.")
    
    
    # Create the base JSON output parser
    base_parser = JsonOutputParser(pydantic_object=Sector)

    # Create the output-fixing parser wrapping the base parser and using an LLM to fix errors
    llm_for_fixing = ChatOpenAI(temperature=0)
    fixing_parser = OutputFixingParser.from_llm(parser=base_parser, llm=llm_for_fixing)

    # Define a rich persona in the system message with added expertise
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp and insightful trader assistant skilled is deriving sector trading insights from news."""
    )

    # Define the human message template with instructions and JSON schema
    human_message = HumanMessagePromptTemplate.from_template(
        """Extract information required.


    News feed:
    {news_feed}

    """
    )

    # Create the chat prompt template
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # Compose the chain: prompt -> LLM -> fixing parser
    llm = ChatOpenAI(temperature=0)
    chain = chat_prompt | llm | fixing_parser
    # all_text = ". ".join(doc.page_content for doc in docs)
    result = chain.invoke({"news_feed": all_text})
    return result

extract_info_from_rss(joined_contents[0])



{'Name': 'Technology Services',
 'Outlook': 'Positive',
 'Catalyst': 'Strong quarterly earnings from major tech companies',
 'Trading_insights': 'Investors are optimistic about the tech sector',
 'Direction_momentum': 'Strength',
 'Sector_vs_market_position': 'Leading'}

In [ ]:
# Define the Pydantic model for structured output
class Sector(BaseModel):
    Name: Literal[
        "Commercial Services",
        "Communications",
        "Consumer Durables",
        "Consumer Non-Durables",
        "Consumer Services",
        "Distribution Services",
        "Electronic Technology",
        "Energy Minerals",
        "Finance",
        "Health Services",
        "Health Technology",
        "Industrial Services",
        "Non-Energy Minerals",
        "Process Industries",
        "Producer Manufacturing",
        "Retail Trade",
        "Technology Services",
        "Transportation",
        "Utilities"
        ] = Field(..., description="Name of the sector")
    Outlook: str = Field(..., description='A sector outlook describes the expected future performance and conditions based on data, trends, and risks.')
    Catalyst: str = Field(..., description="A primary catalyst of a sector is the single most influential force that is expected to drive major change—positive or negative—across an entire industry.")
    Trading_insights: str = Field(..., description="Practical, actionable interpretation of the sector’s expected future conditions—something that can guide investment or trading decisions.")
    Direction_momentum: Literal["Strength", "Weakness", "Volatility", "Rotation in favor", "Rotation out of favor"] = Field(..., description="The direction of momentum of a sector is the prevailing trend in how that sector’s prices, performance, and investor sentiment are moving over a given period.")
    Sector_vs_market_position: Literal['Leading', 'Lagging', 'In Line'] = Field(..., description="Relative positioning of a sector to the overall market is the sector’s performance and strength compared to the broader market, showing whether it is leading, lagging, or moving in line with the market trend.")

In [ ]:

# Create the base JSON output parser
base_parser = JsonOutputParser(pydantic_object=Sector)

# Create the output-fixing parser wrapping the base parser and using an LLM to fix errors
llm_for_fixing = ChatOpenAI(temperature=0)
fixing_parser = OutputFixingParser.from_llm(parser=base_parser, llm=llm_for_fixing)

# Define a rich persona in the system message with added expertise
system_message = SystemMessagePromptTemplate.from_template(
    """You are Ava, a sharp and insightful trader assistant skilled is deriving sector trading insights from news."""
)

# Define the human message template with instructions and JSON schema
human_message = HumanMessagePromptTemplate.from_template(
    """Extract information required.


News feed:
{news_feed}

"""
)

# Create the chat prompt template
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# Compose the chain: prompt -> LLM -> fixing parser
llm = ChatOpenAI(temperature=0)
chain = chat_prompt | llm | fixing_parser
all_text = ". ".join(doc.page_content for doc in docs)
result = chain.invoke({"news_feed": all_text})

# Example news feed
# news = docs[10].page_content

# Invoke the chain
# result = chain.invoke({"news_feed": news})
# print(result)

{'Name': 'Transportation', 'Outlook': 'The transportation sector is facing challenges due to the forecast of a cold snap and massive winter storm sweeping across the country. Airlines are taking preemptive measures by canceling flights and waiving change fees for travelers.', 'Catalyst': 'The primary catalyst for the transportation sector is the weather conditions leading to flight cancellations and disruptions.', 'Trading_insights': 'Investors should monitor how individual airlines manage the situation and adjust their trading strategies accordingly.', 'Direction_momentum': 'Volatility', 'Sector_vs_market_position': 'Lagging'}


In [23]:
rich.print(result)

{
    'Name': 'Transportation',
    'Outlook': 'The transportation sector is facing challenges due to the forecast of a cold snap and massive 
winter storm sweeping across the country. Airlines are taking preemptive measures by canceling flights and waiving 
change fees for travelers.',
    'Catalyst': 'The primary catalyst for the transportation sector is the weather conditions leading to flight 
cancellations and disruptions.',
    'Trading_insights': 'Investors should monitor how individual airlines manage the situation and adjust their 
trading strategies accordingly.',
    'Direction_momentum': 'Volatility',
    'Sector_vs_market_position': 'Lagging'
}

In [40]:
from typing import List
from pydantic import BaseModel, Field
from typing import Literal

def extract_info_from_rss(all_text):



    # Keep your existing Sector model unchanged
    class Sector(BaseModel):
        Name: Literal[
            "Commercial Services",
            "Communications",
            "Consumer Durables",
            "Consumer Non-Durables",
            "Consumer Services",
            "Distribution Services",
            "Electronic Technology",
            "Energy Minerals",
            "Finance",
            "Health Services",
            "Health Technology",
            "Industrial Services",
            "Non-Energy Minerals",
            "Process Industries",
            "Producer Manufacturing",
            "Retail Trade",
            "Technology Services",
            "Transportation",
            "Utilities"
            ] = Field(..., description="Name of the sector")
        Outlook: str = Field(..., description='A sector outlook describes the expected future performance and conditions based on data, trends, and risks.')
        Catalyst: str = Field(..., description="A primary catalyst of a sector is the single most influential force that is expected to drive major change—positive or negative—across an entire industry.")
        Trading_insights: str = Field(..., description="Practical, actionable interpretation of the sector's expected future conditions—something that can guide investment or trading decisions.")
        Direction_momentum: Literal["Strength", "Weakness", "Volatility", "Rotation in favor", "Rotation out of favor"] = Field(..., description="The direction of momentum of a sector is the prevailing trend in how that sector's prices, performance, and investor sentiment are moving over a given period.")
        Sector_vs_market_position: Literal['Leading', 'Lagging', 'In Line'] = Field(..., description="Relative positioning of a sector to the overall market is the sector's performance and strength compared to the broader market, showing whether it is leading, lagging, or moving in line with the market trend.")

    # New wrapper model for multiple sectors
    class MultiSectorAnalysis(BaseModel):
        """Analysis of multiple sectors from news feed."""
        sectors: List[Sector] = Field(
            ...,
            description="List of ALL relevant sectors found in the news (1-8 max). Only include sectors with clear evidence.",
            min_items=1,
            max_items=8
        )
    
    # 1. Create messages WITHOUT partial first
    system_message = SystemMessagePromptTemplate.from_template(
        """You are Ava, a sharp trader assistant. Analyze the FULL news feed and extract **ALL relevant sectors** mentioned.

    CRITICAL RULES:
    - Output ONLY valid JSON matching this schema: {schema}
    - Extract 1-8 sectors with clear news evidence
    - Each sector must use a Name from the enum
    - Fill ALL required fields for each sector
    - No duplicate sectors
    - No extra text outside JSON"""
    )

    human_message = HumanMessagePromptTemplate.from_template(
        """News feed:
    {news_feed}"""
    )

    # 2. Create ChatPromptTemplate
    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

    # 3. NOW partial with schema (after creation)
    base_parser = JsonOutputParser(pydantic_object=MultiSectorAnalysis)
    chat_prompt = chat_prompt.partial(schema=base_parser.get_format_instructions())

    # 4. Build chain
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    chain = chat_prompt | llm.with_structured_output(MultiSectorAnalysis)

    # Invoke
    result = chain.invoke({"news_feed": all_text})
    # results = [chain.invoke({"news_feed": doc}) for doc in docs[:5]]
    return result

extract_info_from_rss(joined_contents[0])



ValidationError: 1 validation error for MultiSectorAnalysis
sectors.1.Direction_momentum
  Input should be 'Strength', 'Weakness', 'Volatility', 'Rotation in favor' or 'Rotation out of favor' [type=literal_error, input_value='In Line', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error

In [ ]:


# Keep your existing Sector model unchanged
class Sector(BaseModel):
    Name: Literal[
        "Commercial Services",
        "Communications",
        "Consumer Durables",
        "Consumer Non-Durables",
        "Consumer Services",
        "Distribution Services",
        "Electronic Technology",
        "Energy Minerals",
        "Finance",
        "Health Services",
        "Health Technology",
        "Industrial Services",
        "Non-Energy Minerals",
        "Process Industries",
        "Producer Manufacturing",
        "Retail Trade",
        "Technology Services",
        "Transportation",
        "Utilities"
        ] = Field(..., description="Name of the sector")
    Outlook: str = Field(..., description='A sector outlook describes the expected future performance and conditions based on data, trends, and risks.')
    Catalyst: str = Field(..., description="A primary catalyst of a sector is the single most influential force that is expected to drive major change—positive or negative—across an entire industry.")
    Trading_insights: str = Field(..., description="Practical, actionable interpretation of the sector's expected future conditions—something that can guide investment or trading decisions.")
    Direction_momentum: Literal["Strength", "Weakness", "Volatility", "Rotation in favor", "Rotation out of favor"] = Field(..., description="The direction of momentum of a sector is the prevailing trend in how that sector's prices, performance, and investor sentiment are moving over a given period.")
    Sector_vs_market_position: Literal['Leading', 'Lagging', 'In Line'] = Field(..., description="Relative positioning of a sector to the overall market is the sector's performance and strength compared to the broader market, showing whether it is leading, lagging, or moving in line with the market trend.")

# New wrapper model for multiple sectors
class MultiSectorAnalysis(BaseModel):
    """Analysis of multiple sectors from news feed."""
    sectors: List[Sector] = Field(
        ...,
        description="List of ALL relevant sectors found in the news (1-8 max). Only include sectors with clear evidence.",
        min_items=1,
        max_items=8
    )

In [11]:
# 1. Create messages WITHOUT partial first
system_message = SystemMessagePromptTemplate.from_template(
    """You are Ava, a sharp trader assistant. Analyze the FULL news feed and extract **ALL relevant sectors** mentioned.

CRITICAL RULES:
- Output ONLY valid JSON matching this schema: {schema}
- Extract 1-8 sectors with clear news evidence
- Each sector must use a Name from the enum
- Fill ALL required fields for each sector
- No duplicate sectors
- No extra text outside JSON"""
)

human_message = HumanMessagePromptTemplate.from_template(
    """News feed:
{news_feed}"""
)

# 2. Create ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# 3. NOW partial with schema (after creation)
base_parser = JsonOutputParser(pydantic_object=MultiSectorAnalysis)
chat_prompt = chat_prompt.partial(schema=base_parser.get_format_instructions())

# 4. Build chain
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = chat_prompt | llm.with_structured_output(MultiSectorAnalysis)

# Invoke
# result = chain.invoke({"news_feed": all_text})

In [16]:
results = [chain.invoke({"news_feed": doc}) for doc in docs[:5]]

In [17]:
import rich
rich.print(results)

[
    MultiSectorAnalysis(
        sectors=[
            Sector(
                Name='Technology Services',
                Outlook='Positive growth expected due to strong earnings performance.',
                Catalyst="Microsoft's quarterly earnings beat expectations, driving market cap growth.",
                Trading_insights="Investors may consider increasing positions in technology stocks following 
Microsoft's success.",
                Direction_momentum='Strength',
                Sector_vs_market_position='Leading'
            )
        ]
    ),
    MultiSectorAnalysis(
        sectors=[
            Sector(
                Name='Technology Services',
                Outlook='The technology services sector is expected to face challenges due to significant losses 
reported by major companies.',
                Catalyst='The primary catalyst is the financial performance of leading tech firms, particularly in 
emerging technologies like virtual reality.',
                Trading_insights='Investors should be cautious as losses may indicate broader issues within the 
tech sector, impacting stock prices.',
                Direction_momentum='Weakness',
                Sector_vs_market_position='Lagging'
            )
        ]
    ),
    MultiSectorAnalysis(
        sectors=[
            Sector(
                Name='Commercial Services',
                Outlook='The imposition of 25% duties on imports from India may lead to increased costs for U.S. 
businesses relying on Indian goods, impacting their profitability and operational strategies.',
                Catalyst="The U.S. government's decision to impose tariffs on imports from India is a significant 
catalyst that could reshape trade dynamics and influence market behavior.",
                Trading_insights='Investors should monitor companies heavily reliant on Indian imports, as they may
face margin pressures and potential shifts in supply chains.',
                Direction_momentum='Weakness',
                Sector_vs_market_position='Lagging'
            )
        ]
    ),
    MultiSectorAnalysis(
        sectors=[
            Sector(
                Name='Commercial Services',
                Outlook='The commercial services sector is expected to face challenges due to limited trade 
agreements.',
                Catalyst='The slow pace of trade deal negotiations, particularly with the EU, is a significant 
catalyst affecting this sector.',
                Trading_insights='Investors should be cautious as the lack of trade deals may hinder growth in 
commercial services.',
                Direction_momentum='Weakness',
                Sector_vs_market_position='Lagging'
            )
        ]
    ),
    MultiSectorAnalysis(
        sectors=[
            Sector(
                Name='Finance',
                Outlook='The finance sector is expected to remain stable as interest rates are held steady, 
providing a predictable environment for lending and investment.',
                Catalyst="The Federal Reserve's decision to keep interest rates unchanged is a primary catalyst 
influencing the finance sector.",
                Trading_insights='Investors may find opportunities in financial institutions that benefit from 
stable interest rates, such as banks and mortgage lenders.',
                Direction_momentum='Strength',
                Sector_vs_market_position='Leading'
            )
        ]
    )
]

['Wed, 30 Jul 2025 22:27:00 GMT: Based on its post-market trading, Microsoft has become the world\'s second $4 trillion company following quarterly earnings.  .Wed, 30 Jul 2025 20:20:49 GMT: Meta’s Reality Labs posts $4.53 billion loss in second quarter.Thu, 31 Jul 2025 04:28:17 GMT: U.S. President Donald Trump on Wednesday announced 25% duties on imports from India and an unspecified \\penalty.\\"".Thu, 31 Jul 2025 03:49:02 GMT: The U.S. has managed to make only eight deals in 120 days, including one with the 27-member European Union..Wed, 30 Jul 2025 20:08:27 GMT: A divided Federal Reserve on Wednesday voted to keep its benchmark interest rate steady, with two top officials dissenting..Wed, 30 Jul 2025 19:54:30 GMT: \\That meeting will have fireworks; today was just sparklers,\\" one investor said.".Thu, 31 Jul 2025 02:38:54 GMT: This deal would mean the duties will be lowered from the 25% that Trump had threatened in his \\tariff letter\\" to Seoul earlier this month.".Thu, 31 Jul 2

In [44]:
# Wrapper for multiple sectors
class SectorAnalysis(BaseModel):
    """Multiple sector analysis from news."""
    sectors: List[Sector] = Field(
        description="All relevant sectors extracted from news (1-8 max)",
        min_items=1,
        max_items=8
    )

# Now it works!
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = chat_prompt | llm.with_structured_output(SectorAnalysis)

result = chain.invoke({"news_feed": joined_contents[10]})
print(f"Found {len(result.sectors)} sectors:")
for s in result.sectors:
    print(f"  - {s.Name}: {s.Direction_momentum} ({s.Sector_vs_market_position})")

Found 7 sectors:
  - Consumer Services: Weakness (Lagging)
  - Finance: Strength (Leading)
  - Technology Services: Strength (Leading)
  - Industrial Services: Strength (Leading)
  - Consumer Non-Durables: Weakness (Lagging)
  - Transportation: Strength (Leading)
  - Utilities: Volatility (In Line)


In [45]:
rich.print(result)

SectorAnalysis(
    sectors=[
        Sector(
            Name='Consumer Services',
            Outlook='The fast-casual dining sector is experiencing a slowdown as consumer spending tightens, 
impacting chains like Chipotle and Cava.',
            Catalyst='Consumer spending trends and economic conditions affecting dining out.',
            Trading_insights='Investors should monitor consumer sentiment and spending patterns as they may 
influence the performance of fast-casual dining stocks.',
            Direction_momentum='Weakness',
            Sector_vs_market_position='Lagging'
        ),
        Sector(
            Name='Finance',
            Outlook='Mortgage rates have dropped, leading to increased demand for refinancing, which could benefit 
financial institutions involved in mortgage lending.',
            Catalyst='Declining mortgage rates stimulating refinancing activity.',
            Trading_insights='Investors might consider financial stocks that are heavily involved in mortgage 
lending as potential beneficiaries of the current rate environment.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        ),
        Sector(
            Name='Technology Services',
            Outlook='The technology sector, particularly AI infrastructure and cybersecurity, is seeing strong 
growth driven by increased demand.',
            Catalyst='Rising interest in AI and cybersecurity solutions.',
            Trading_insights='Investors should focus on tech companies that are innovating in AI and cybersecurity 
as they are likely to outperform in the current market.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        ),
        Sector(
            Name='Industrial Services',
            Outlook='The industrial sector is showing resilience with companies identifying wide moat stocks, 
indicating strong competitive advantages.',
            Catalyst='Identifying companies with strong market positions and competitive advantages.',
            Trading_insights='Investors should look for industrial stocks with solid fundamentals and competitive 
moats as potential long-term investments.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        ),
        Sector(
            Name='Consumer Non-Durables',
            Outlook='The consumer non-durables sector is facing challenges as inflation impacts consumer spending, 
particularly in essential goods.',
            Catalyst='Inflation affecting consumer purchasing power and spending habits.',
            Trading_insights='Investors should be cautious with consumer non-durables stocks as rising costs may 
squeeze margins and consumer demand.',
            Direction_momentum='Weakness',
            Sector_vs_market_position='Lagging'
        ),
        Sector(
            Name='Transportation',
            Outlook='The transportation sector is recovering as airlines revive routes and expand international 
flights post-pandemic.',
            Catalyst='Increased travel demand and airline route expansions.',
            Trading_insights='Investors may find opportunities in airline stocks as travel demand rebounds, but 
should remain aware of potential economic headwinds.',
            Direction_momentum='Strength',
            Sector_vs_market_position='Leading'
        ),
        Sector(
            Name='Utilities',
            Outlook='Utilities are facing regulatory scrutiny and potential changes in tariffs, impacting their 
operational strategies.',
            Catalyst='Regulatory changes and tariff implications affecting utility companies.',
            Trading_insights='Investors should monitor regulatory developments that could impact utility stocks and
their profitability.',
            Direction_momentum='Volatility',
            Sector_vs_market_position='In Line'
        )
    ]
)

In [44]:
for sector in result.sectors:
    print(f"\n📊 {sector.Name}")
    print(f"   Momentum: {sector.Direction_momentum}")
    print(f"   vs Market: {sector.Sector_vs_market_position}")
    print(f"   Catalyst: {sector.Catalyst[:100]}...")
    print(f"   Trading: {sector.Trading_insights[:100]}...")


📊 Finance
   Momentum: Strength
   vs Market: Leading
   Catalyst: The anticipated growth in private credit from $3.4 trillion in 2025 to $4.9 trillion by 2029 is a ke...
   Trading: Investors should consider diversifying into private credit funds or related financial instruments to...

📊 Technology Services
   Momentum: Strength
   vs Market: Leading
   Catalyst: The upcoming IPOs and advancements in AI technology are driving investor interest and market momentu...
   Trading: Focus on tech stocks with strong growth potential, especially those involved in AI and cloud computi...

📊 Health Technology
   Momentum: Strength
   vs Market: In Line
   Catalyst: Insights from executives at the JPMorgan Healthcare Conference highlight ongoing advancements and in...
   Trading: Consider investing in biotech and health tech companies that are presenting innovative solutions at ...

📊 Consumer Non-Durables
   Momentum: Weakness
   vs Market: Lagging
   Catalyst: China's record-low birth rate is

In [45]:
strong_sectors = [s for s in result.sectors if s.Direction_momentum == "Strength"]
print(f"🚀 Top buys: {[s.Name for s in strong_sectors]}")

🚀 Top buys: ['Finance', 'Technology Services', 'Health Technology', 'Retail Trade', 'Industrial Services']


In [46]:
leaders = sorted(result.sectors, key=lambda s: ["Lagging", "In Line", "Leading"].index(s.Sector_vs_market_position))
print("🏆 Market leaders:", [s.Name for s in leaders[-3:]])

🏆 Market leaders: ['Finance', 'Technology Services', 'Industrial Services']


In [47]:
signals = {}
for s in result.sectors:
    key = f"{s.Name} ({s.Direction_momentum})"
    signals[key] = s.Sector_vs_market_position

print("📈 Trading signals:")
for signal, pos in signals.items():
    emoji = "🟢" if "Strength" in signal else "🔴"
    print(f"{emoji} {signal} → {pos}")

📈 Trading signals:
🟢 Finance (Strength) → Leading
🟢 Technology Services (Strength) → Leading
🟢 Health Technology (Strength) → In Line
🔴 Consumer Non-Durables (Weakness) → Lagging
🔴 Energy Minerals (Weakness) → Lagging
🟢 Retail Trade (Strength) → In Line
🔴 Transportation (Weakness) → Lagging
🟢 Industrial Services (Strength) → Leading


In [50]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI

class TradingEntity(BaseModel):
    """Single trading signal."""
    name: str = Field(..., description="Sector or company name (exact match from news)")
    note: str = Field("", description="Optional caution/note like '(with caution)'")

class TradingSignals(BaseModel):
    """Bullish vs Bearish signals from news."""
    bullish: List[TradingEntity] = Field(
        min_items=1, max_items=8,
        description="Bullish sectors/companies with growth momentum"
    )
    bearish: List[TradingEntity] = Field(
        min_items=1, max_items=8, 
        description="Bearish sectors/companies with risks/weakness"
    )

# Prompt - NO {schema} variable needed!
system_message = SystemMessagePromptTemplate.from_template(
    """You are Ava, elite trader scanning news for actionable signals.

From the news, extract **BULLISH** (growth/strength) vs **BEARISH** (risks/weakness) sectors/companies.

Output as JSON array of exact names from news. Use "note" for cautions.

Examples:
- Bullish: "Private credit sector", note="with caution"
- Bearish: "European carmakers"

Rules:
- 3-8 per list maximum
- Only signals with clear news evidence
- Companies + sectors both OK"""
)

human_message = HumanMessagePromptTemplate.from_template(
    """News feed:
{news_feed}"""
)

# Create prompt (variables: only {news_feed})
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# with_structured_output AUTOMATICALLY injects schema!
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = chat_prompt | llm.with_structured_output(TradingSignals)

# Run
all_text = ". ".join(doc.page_content for doc in docs)
result = chain.invoke({"news_feed": all_text})

# Pretty print EXACTLY your format
print("\n**Bullish Sectors/Companies**:")
for entity in result.bullish:
    note = f" ({entity.note})" if entity.note else ""
    print(f"   - {entity.name}{note}")

print("\n**Bearish Sectors/Companies**:")
for entity in result.bearish:
    note = f" ({entity.note})" if entity.note else ""
    print(f"   - {entity.name}{note}")


**Bullish Sectors/Companies**:
   - Private credit sector (with caution)
   - Czech-based defense firm Czechoslovak Group (CSG)
   - Nvidia
   - Industrious
   - Google
   - Procter & Gamble
   - AI foundation model developers
   - U.S. airlines (with caution)

**Bearish Sectors/Companies**:
   - European carmakers
   - Indigo
   - China's tech giants
   - Reliance Industries
   - General Motors
   - U.S. Treasurys
   - China's homebuilders
   - Spirit Airlines


In [51]:
rich.print(result)

TradingSignals(
    bullish=[
        TradingEntity(name='Private credit sector', note='with caution'),
        TradingEntity(name='Czech-based defense firm Czechoslovak Group (CSG)', note=''),
        TradingEntity(name='Nvidia', note=''),
        TradingEntity(name='Industrious', note=''),
        TradingEntity(name='Google', note=''),
        TradingEntity(name='Procter & Gamble', note=''),
        TradingEntity(name='AI foundation model developers', note=''),
        TradingEntity(name='U.S. airlines', note='with caution')
    ],
    bearish=[
        TradingEntity(name='European carmakers', note=''),
        TradingEntity(name='Indigo', note=''),
        TradingEntity(name="China's tech giants", note=''),
        TradingEntity(name='Reliance Industries', note=''),
        TradingEntity(name='General Motors', note=''),
        TradingEntity(name='U.S. Treasurys', note=''),
        TradingEntity(name="China's homebuilders", note=''),
        TradingEntity(name='Spirit Airlines', note='')
    ]
)

In [52]:
from typing import Literal
from pydantic import BaseModel, Field
from typing import Optional

class TradingEntity(BaseModel):
    """Detailed trading signal with conviction & catalysts."""
    name: str = Field(..., description="Exact sector/company name from news")
    
    # Trading conviction (strength of signal)
    conviction: Literal["High", "Medium", "Low"] = Field(
        ..., 
        description="Signal strength: High=strong evidence, Low=emerging/weak"
    )
    
    # Key catalyst driving the signal
    catalyst: str = Field(
        ..., 
        description="Primary news catalyst (1 sentence max)"
    )
    
    # Optional caution/risk
    note: Optional[str] = Field(
        None, 
        description="Risks/cautions like '(regulatory risk)' or '(overbought)'"
    )
    
    # Price action suggestion
    action: Literal["Buy", "Sell", "Watch", "Avoid"] = Field(
        ..., 
        description="Clear trading action"
    )

class TradingSignals(BaseModel):
    """Detailed bullish/bearish signals."""
    bullish: List[TradingEntity] = Field(min_items=1, max_items=10)
    bearish: List[TradingEntity] = Field(min_items=1, max_items=10)

In [55]:
system_message = SystemMessagePromptTemplate.from_template(
    """Ava: Elite trader extracting HIGH-CONVICTION signals from news.

For each signal provide:
- name: Exact sector/company from news
- conviction: High/Medium/Low (evidence strength)  
- catalyst: 1-sentence key driver
- action: Buy/Sell/Watch/Avoid
- note: Risks/cautions (optional)

Examples:
- High conviction Buy: "Private credit", catalyst="3.4T→4.9T growth", note="regulatory risk"

Only clear signals."""
)

# Same chain setup
chain = chat_prompt | llm.with_structured_output(TradingSignals)
# Run
all_text = ". ".join(doc.page_content for doc in docs)
result = chain.invoke({"news_feed": all_text})

rich.print(result)

TradingSignals(
    bullish=[
        TradingEntity(
            name='Private credit sector',
            conviction='High',
            catalyst='Private credit is expected to grow from $3.4 trillion in 2025 to an estimated $4.9 trillion 
by 2029.',
            note='with caution',
            action='Buy'
        ),
        TradingEntity(
            name='Czech-based defense firm Czechoslovak Group (CSG)',
            conviction='High',
            catalyst='Shares of Czech-based defense firm Czechoslovak Group (CSG) soared over 31% on its stock 
market debut on Friday.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            name='Nvidia',
            conviction='High',
            catalyst='Intel has gotten a boost due to optimism around its latest server CPUs and following 
investments from the U.S. government and Nvidia.',
            note=None,
            action='Buy'
        ),
        TradingEntity(
            name='AI foundation model developers',
            conviction='Medium',
            catalyst='AI foundation model developers are still experimenting with business models, but investors 
will likely push for returns this year.',
            note=None,
            action='Watch'
        ),
        TradingEntity(
            name="India's live events space",
            conviction='Medium',
            catalyst='India saw a 17% rise in the live events space in 2025, marking a boom in demand for 
experiences.',
            note=None,
            action='Watch'
        ),
        TradingEntity(
            name='Google',
            conviction='Medium',
            catalyst="Google is enabling retail's transition into agentic commerce, rewriting how we shop online.",
            note=None,
            action='Watch'
        ),
        TradingEntity(
            name='Procter & Gamble',
            conviction='Medium',
            catalyst="Procter & Gamble topped Wall Street's estimates for its quarterly earnings, but the company's
revenue was weaker than expected.",
            note='watch for revenue trends',
            action='Watch'
        )
    ],
    bearish=[
        TradingEntity(
            name='European carmakers',
            conviction='High',
            catalyst="Shares of some of Europe's biggest carmakers fell on Monday morning.",
            note=None,
            action='Sell'
        ),
        TradingEntity(
            name='Indigo',
            conviction='Medium',
            catalyst="Weak currency, labour reforms and passenger compensation costs dragged Indigo's earnings.",
            note=None,
            action='Sell'
        ),
        TradingEntity(
            name='Intel',
            conviction='Medium',
            catalyst='Intel reported fourth-quarter earnings Thursday that beat Wall Street expectations but 
offered soft guidance for the current quarter.',
            note='watch for guidance',
            action='Watch'
        ),
        TradingEntity(
            name="China's economic growth",
            conviction='High',
            catalyst="China's economic growth slowed to the weakest level in nearly three years in the fourth 
quarter as retail sales fell to the lowest level since late 2022.",
            note=None,
            action='Avoid'
        ),
        TradingEntity(
            name='Reliance Industries',
            conviction='Medium',
            catalyst='Slowdown in sales growth of Reliance Retail has led brokerages to cut target prices on 
Reliance Industries.',
            note=None,
            action='Sell'
        ),
        TradingEntity(
            name='U.S. Treasurys',
            conviction='Medium',
            catalyst="U.S. Treasurys and other countries' government bonds sold off on Tuesday, after U.S. 
President Donald Trump threatened new tariffs on various European allies.",
            note=None,
            action='Avoid'
        ),
        TradingEntity(
            name="China's tech sect

In [56]:
def print_signals(result: TradingSignals):
    print("\n🚀 HIGH-CONVICTION BUYS:")
    high_buys = [e for e in result.bullish if e.conviction == "High" and e.action == "Buy"]
    for e in high_buys:
        print(f"  💰 {e.name}")
        print(f"     {e.catalyst}")
        if e.note: print(f"     ⚠️  {e.note}")
    
    print("\n📉 HIGH-CONVICTION SELLS:")
    high_sells = [e for e in result.bearish if e.conviction == "High" and e.action == "Sell"]
    for e in high_sells:
        print(f"  💸 {e.name}")
        print(f"     {e.catalyst}")

print_signals(result)


🚀 HIGH-CONVICTION BUYS:
  💰 Private credit sector
     Private credit is expected to grow from $3.4 trillion in 2025 to an estimated $4.9 trillion by 2029.
     ⚠️  with caution
  💰 Czech-based defense firm Czechoslovak Group (CSG)
     Shares of Czech-based defense firm Czechoslovak Group (CSG) soared over 31% on its stock market debut on Friday.
  💰 Nvidia
     Intel has gotten a boost due to optimism around its latest server CPUs and following investments from the U.S. government and Nvidia.

📉 HIGH-CONVICTION SELLS:
  💸 European carmakers
     Shares of some of Europe's biggest carmakers fell on Monday morning.


In [57]:
bull_score = sum(1 if e.conviction == "High" else 0.5 if e.conviction == "Medium" else 0 
                for e in result.bullish)
bear_score = sum(1 if e.conviction == "High" else 0.5 if e.conviction == "Medium" else 0 
                for e in result.bearish)

market_bias = "🟢 BULLISH" if bull_score > bear_score * 1.2 else "🔴 BEARISH"
print(f"\n📊 Net Bias: {market_bias} (Bull: {bull_score:.1f}, Bear: {bear_score:.1f})")


📊 Net Bias: 🔴 BEARISH (Bull: 5.0, Bear: 4.5)


In [58]:
buy_list = [e.name for e in result.bullish if e.action == "Buy"]
watch_list = [e.name for e in result.bullish if e.action == "Watch"]
sell_list = [e.name for e in result.bearish if e.action == "Sell"]

print(f"🛒 BUY: {', '.join(buy_list)}")
print(f"👀 WATCH: {', '.join(watch_list)}")
print(f"📉 SELL: {', '.join(sell_list)}")

🛒 BUY: Private credit sector, Czech-based defense firm Czechoslovak Group (CSG), Nvidia
👀 WATCH: AI foundation model developers, India's live events space, Google, Procter & Gamble
📉 SELL: European carmakers, Indigo, Reliance Industries


In [59]:
# Convert to DataFrame
def signals_to_df(signals: TradingSignals) -> pd.DataFrame:
    # Flatten bullish
    bullish_df = pd.DataFrame([{
        'signal_type': 'BULLISH',
        'name': e.name,
        'conviction': e.conviction,
        'catalyst': e.catalyst,
        'note': e.note or '',
        'action': e.action,
        'score': {'High': 3, 'Medium': 2, 'Low': 1}[e.conviction]
    } for e in signals.bullish])
    
    # Flatten bearish  
    bearish_df = pd.DataFrame([{
        'signal_type': 'BEARISH',
        'name': e.name,
        'conviction': e.conviction,
        'catalyst': e.catalyst,
        'note': e.note or '',
        'action': e.action,
        'score': -{'High': 3, 'Medium': 2, 'Low': 1}[e.conviction]  # Negative for bearish
    } for e in signals.bearish])
    
    return pd.concat([bullish_df, bearish_df], ignore_index=True)

# Create DataFrame
df = signals_to_df(result)
print(df)

# Pretty display
print("\n📊 TRADING SIGNALS DATAFRAME:")
print(df.to_string(index=False))

   signal_type                                               name conviction  \
0      BULLISH                              Private credit sector       High   
1      BULLISH  Czech-based defense firm Czechoslovak Group (CSG)       High   
2      BULLISH                                             Nvidia       High   
3      BULLISH                     AI foundation model developers     Medium   
4      BULLISH                          India's live events space     Medium   
5      BULLISH                                             Google     Medium   
6      BULLISH                                   Procter & Gamble     Medium   
7      BEARISH                                 European carmakers       High   
8      BEARISH                                             Indigo     Medium   
9      BEARISH                                              Intel     Medium   
10     BEARISH                            China's economic growth       High   
11     BEARISH                          

In [60]:
df

,signal_type,name,conviction,catalyst,note,action,score
0,BULLISH,Private credit sector,High,Private credit is expected to grow from $3.4 t...,with caution,Buy,3
1,BULLISH,Czech-based defense firm Czechoslovak Group (CSG),High,Shares of Czech-based defense firm Czechoslova...,,Buy,3
2,BULLISH,Nvidia,High,Intel has gotten a boost due to optimism aroun...,,Buy,3
3,BULLISH,AI foundation model developers,Medium,AI foundation model developers are still exper...,,Watch,2
4,BULLISH,India's live events space,Medium,India saw a 17% rise in the live events space ...,,Watch,2
5,BULLISH,Google,Medium,Google is enabling retail's transition into ag...,,Watch,2
6,BULLISH,Procter & Gamble,Medium,Procter & Gamble topped Wall Street's estimate...,watch for revenue trends,Watch,2
7,BEARISH,European carmakers,High,Shares of some of Europe's biggest carmakers f...,,Sell,-3
8,BEARISH,Indigo,Medium,"Weak currency, labour reforms and passenger co...",,Sell,-2
9,BEARISH,Intel,Medium,Intel reported fourth-quarter earnings Thursda...,watch for guidance,Watch,-2


In [24]:
batch_inputs = []
for doc in docs:
    metadata_text = "\n".join(f"{k}: {v}" for k, v in (doc.metadata or {}).items())
    news_feed = f"{doc.page_content}\n\nMetadata:\n{metadata_text}"
    batch_inputs.append({"news_feed": news_feed})

batch_inputs

[{'news_feed': 'Private credit is expected to grow from $3.4 trillion in 2025 to an estimated $4.9 trillion by 2029. Wall Street is starting to raise alarms about the risks. \n\nMetadata:\nlink: https://www.cnbc.com/2026/01/23/wall-street-private-credit-risk-rising.html\nguid: 108255689\ntype: cnbcnewsstory\nid: 108255689\nsponsored: False\npubDate: Fri, 23 Jan 2026 12:48:57 GMT'},
 {'news_feed': 'Sen. Elizabeth Warren seized on contradictions within the Trump administration when it comes to his push to make credit cards more affordable for Americans.\n\nMetadata:\nlink: https://www.cnbc.com/2026/01/23/warren-cfpb-director-letter-undermining-trump-credit-card-affordability.html\nguid: 108256452\ntype: cnbcnewsstory\nid: 108256452\nsponsored: False\npubDate: Fri, 23 Jan 2026 19:00:03 GMT'},
 {'news_feed': 'Greenland has the eighth-largest rare earth reserves in the world at 1.5 million metric tons, according to 2024 data from the U.S. Geological Survey. \n\nMetadata:\nlink: https://www.

In [25]:
result = chain.batch(batch_inputs)
rich.print(result)

[
    {
        'Name': 'Finance',
        'Outlook': 'Private credit sector projected to increase from $3.4 trillion in 2025 to around $4.9 trillion 
by 2029.',
        'Catalyst': 'Concerns raised on Wall Street regarding associated risks of private credit sector growth.',
        'Trading_insights': 'Significant expansion expected in private credit sector over the next few years.',
        'Direction_momentum': 'Strength',
        'Sector_vs_market_position': 'Leading'
    },
    {
        'Name': 'Consumer Services',
        'Outlook': 'Highlighting contradictions within the Trump administration regarding efforts to make credit 
cards more affordable for Americans',
        'Catalyst': "Letter from the Consumer Financial Protection Bureau (CFPB) director undermining Trump's 
initiative on credit card affordability",
        'Trading_insights': "Potential conflict or lack of alignment within the administration's approach to credit
card affordability",
        'Direction_momentum': 'Strength',
        'Sector_vs_market_position': 'Leading'
    },
    {
        'Name': 'Non-Energy Minerals',
        'Outlook': 'Positive',
        'Catalyst': 'Greenland having the eighth-largest rare earth reserves globally',
        'Trading_insights': 'Valuable information for traders interested in the rare earth minerals sector',
        'Direction_momentum': 'Strength',
        'Sector_vs_market_position': 'Leading'
    },
    {
        'Name': 'Technology Services',
        'Outlook': 'Positive',
        'Catalyst': 'Upcoming IPO',
        'Trading_insights': 'Potential investment opportunity',
        'Direction_momentum': 'Strength',
        'Sector_vs_market_position': 'Leading'
    },
    {
        'Name': 'Finance',
        'Outlook': 'Positive',
        'Catalyst': 'Easing geopolitical fears',
        'Trading_insights': 'Stocks rose for a second consecutive session',
        'Direction_momentum': 'Strength',
        'Sector_vs_market_position': 'Leading'
    },
    {
        'Name': '',
        'Outlook': '',
        'Catalyst': '',
        'Trading_insights': '',
        'Direction_momentum': '',
        'Sector_vs_market_position': ''
    },
    {
        'Name': 'Electronic Technology',
        'Outlook': 'Positive',
        'Catalyst': "Intel's stock rally",
        'Trading_insights': 'Potential trading opportunities within the semiconductor sector',
        'Direction_momentum': 'Strength',
        'Sector_vs_market_position': 'Leading'
    },
    {
        'Name': 'Energy Minerals',
        'Outlook': 'The recent civil unrest in Iran has impacted the oil markets, despite Iran not being a major 
producer of oil.',
        'Catalyst': 'The reason behind this influence is detailed in the news article available at the following 
link: [Iran Protests: Why Oil Markets Care So Much About the 
Country](https://www.cnbc.com/2026/01/23/iran-protests-why-oil-markets-care-so-much-about-the-country.html).',
        'Trading_insights': 'The civil unrest in Iran has caused fluctuations in the oil markets, affecting trading
decisions.',
        'Direction_momentum': 'Volatility',
        'Sector_vs_market_position': 'Leading'
    },
    {
        'Name': 'Technology Services',
        'Outlook': 'Positive outlook for the sector with expected growth in the upcoming quarter.',
        'Catalyst': 'Introduction of new innovative technology driving sector growth.',
        'Trading_insights': 'Investors are advised to consider long-term positions in tech companies.',
        'Direction_momentum': 'Strength',
        'Sector_vs_market_position': 'Leading'
    },
    {
        'Name': 'Finance',
        'Outlook': 'Positive',
        'Catalyst': 'Technological advancements in digital finance',
        'Trading_insights': 'Recommendation to invest in fintech companies',
        'Direction_momentum': 'Strength',
        'Sector_vs_market_position': 'Leading'
    },
    {
        'Name': 'Transportation',
        'Outlook': 'The transportation secto